In [5]:
from jax.experimental import sparse
import jax
from jaxlcp import lcp
from jaxlcp.fischer import build_G, T
import scipy
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)

In [6]:
M = jnp.array([[1,-1,-1,-1],[-1,1,-1,-1],[1,1,2,0],[1,1,0,2]], dtype=jnp.float32)
q = jnp.array([3,5,-9,-5])
M = sparse.BCOO.fromdense(M)
x = lcp(M,q, maxit=1, spsolve_tol=1e-15)
x

E0716 03:44:00.327457  399609 pjrt_stream_executor_client.cc:2985] Execution of replica 0 failed: INTERNAL: CustomCall failed: Singular matrix in linear solve.


XlaRuntimeError: INTERNAL: CustomCall failed: Singular matrix in linear solve.

In [9]:
n = M.shape[0]
w =  jnp.column_stack([jnp.ones(n),jnp.zeros(n)])
tau = 1/2*jnp.minimum(jnp.maximum(w[:,0],0), jnp.linalg.norm(T(M,q,w)))
w = w.at[:,1].set(jnp.where(3*jnp.abs(w[:,1])<=tau,
                                    w[:,1]+jnp.sign(M@w[:,0]-w[:,1]+q)*tau,
                                    w[:,1]))
mu = jnp.where(jnp.all(w != 0, axis=1), -1 + w[:, 0] / jnp.sqrt(w[:, 0]**2 + w[:, 1]**2), -1)
nu = jnp.where(jnp.all(w != 0, axis=1), -1 + w[:, 1] / jnp.sqrt(w[:, 0]**2 + w[:, 1]**2), 0)
G = build_G(M, mu, nu)

sparse.linalg.spsolve(G.data, G.indices, G.indptr, G @ w.flatten('F') - T(M, q, w))

w_hat =jnp.linalg.solve(G.todense(), G @ w.flatten('F') - T(M, q, w))

E0716 03:44:25.861896  399609 pjrt_stream_executor_client.cc:2985] Execution of replica 0 failed: INTERNAL: CustomCall failed: Singular matrix in linear solve.


XlaRuntimeError: INTERNAL: CustomCall failed: Singular matrix in linear solve.

In [8]:
scipy.sparse.linalg.spsolve(G, G @ w.flatten('F') - T(M, q, w))

ValueError: unrecognized csc_matrix constructor usage

In [34]:
G@w_hat-(G @ w.flatten('F') - T(M, q, w))

Array([ 0.00000000e+00,  8.88178420e-16,  0.00000000e+00,  8.88178420e-16,
       -2.77555756e-17, -6.93889390e-17, -5.55111512e-17, -1.38777878e-17],      dtype=float64)

In [27]:
jnp.linalg.det(G.todense())

Array(12.77777434, dtype=float64)

In [39]:
bad = jnp.array([False, True, True, False])
print(M[~bad][:,~bad].todense())
bad = jnp.arange(4)[bad]
bad = jnp.column_stack([bad, jnp.arange(len(bad))])
bad

[[ 1. -1.]
 [ 1.  2.]]


Array([[1, 0],
       [2, 1]], dtype=int64)

In [19]:
new_index = M.indices

In [46]:
sparse.BCOO((M.data[:4],new_index[~jnp.isin(new_index, bad[:,0]).any(axis=1),:]),
            shape=(4,4)).todense()

Array([[ 1.,  0.,  0., -1.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [-1.,  0.,  0., -1.]], dtype=float64)

In [42]:
M.data

Array([ 1., -1., -1., -1., -1.,  1., -1., -1.,  1.,  1.,  2.,  1.,  1.,
        2.], dtype=float64)

In [4]:
build_G(M,q,q)

BCOO(float64[4, 4], nse=14)


BCOO(float64[4, 8], nse=20)
BCOO(float64[4, 8], nse=8)


TypeError: lax.concatenate requires arguments to have the same dtypes, got int32, int64. (Tip: jnp.concatenate is a similar function that does automatic type promotion on inputs).

In [11]:
eye(4)

BCOO(float64[4, 4], nse=4)

In [16]:
M

BCOO(float64[4, 4], nse=14)

In [7]:
x, y

(Array([1.9999998 , 0.9999999 , 3.        , 0.99999964], dtype=float32),
 Array([-4.7748472e-12,  3.3717458e-07, -3.3717515e-07, -5.0576216e-07],      dtype=float32))

In [11]:
key = jax.random.PRNGKey(1)
key, *subkeys = jax.random.split(key,3)

n = 1000
q = jax.random.uniform(subkeys[0],(n,))
M = sparse.random_bcoo(subkeys[1],(n,n))

In [21]:
@jax.jit
def foo(M):
    return sparse.bcoo_concatenate([M, eye(n)], dimension=1)

foo(M)

BCOO(float64[1000, 2000], nse=201000)

In [24]:
jnp.max(jnp.abs(M@x + q -y))

Array(5.9604645e-08, dtype=float32)